In [1]:
!pwd

/kaggle/working


In [2]:
import os
import torch

if os.path.exists("/kaggle/working/EmpDialogue_RecEC"):
    %cd EmpDialogue_RecEC/

if os.path.isdir("/kaggle/working/EmpDialogue_RecEC"):
    !git pull origin enhancements
else:
    !git clone -b enhancements https://github.com/saunack/EmpDialogue_RecEC.git
    # or just add "EmpDialogue_RecEC/" in front of all files and remove this cd
    %cd EmpDialogue_RecEC

/kaggle/working/EmpDialogue_RecEC
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Total 6 (delta 5), reused 6 (delta 5), pack-reused 0
Unpacking objects: 100% (6/6), 506 bytes | 253.00 KiB/s, done.
From https://github.com/saunack/EmpDialogue_RecEC
 * branch            enhancements -> FETCH_HEAD
   f99f0d7..f1f7782  enhancements -> origin/enhancements
Updating f99f0d7..f1f7782
Fast-forward
 emotion/config_data.py |  2 +-
 soft_gen/main.py       | 12 ++++++------
 2 files changed, 7 insertions(+), 7 deletions(-)


In [3]:
!bash setup.sh

Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.3/431.3 kB 4.7 MB/s eta 0:00:00a 0:00:01
Creating directories...
Archive:  /kaggle/working/EmpDialogue_RecEC/resources/glove.6B.zip
Getting trained emotion recognition model...
On Kaggle
Setup complete.


In [4]:
# [EMOTION MODEL] Run if emotion recognition model best checkpoint is unavailable
# !bash bash/run_emotion.sh --glove /resources/glove.6B.300d.txt --gpu_id 0
# !cp outputs/emotion/best_emotion.pt models/emotion/best_emotion.pt

[2024-03-10 02:18:26,327 INFO] Vocab size: 38622
[2024-03-10 02:18:26,327 INFO] EmotionVocab Size: 32
[2024-03-10 02:18:32,802 INFO] Training data size: 40252
[2024-03-10 02:18:32,803 INFO] Valid data size: 5736
[2024-03-10 02:18:32,803 INFO] Test data size: 5257
/opt/conda/lib/python3.10/site-packages/torch/utils/data/sampler.py:64: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn("`data_source` argument is not used and will be removed in 2.2.0."
[2024-03-10 02:18:58,507 INFO] saving emotion model...
[2024-03-10 02:18:58,645 INFO] done!
[2024-03-10 02:18:58,646 INFO] epoch: 0 | step: 0 | emotion_loss: 3.5997 | cause_loss: 1.0941 | emotion_accu: 0.0267 | loss: 4.6939
[2024-03-10 02:18:59,828 INFO] saving emotion model...
[2024-03-10 02:18:59,962 INFO] done!
[2024-03-10 02:18:59,963 INFO] epoch: 0 | step: 100 | emotion_loss: 3.5769 | cause_loss: 0.7624 | emotion_accu: 0.0343 | loss: 4.

In [5]:
# !wget -P "resources" https://github.com/Tiiiger/bert_score/blob/master/bert_score/rescale_baseline/en/roberta-base.tsv

In [4]:
torch.cuda.empty_cache()

mode = 3  # 1: train, 2: test, 3: both

if mode == 1 or mode == 3:
    !bash bash/run_generation.sh \
             --glove "resources/glove.6B.300d.txt" \
             --gpu_id 0 \
             --mode soft \
             --roberta roberta-large \
             --baseline "resources/roberta-large.tsv" \
             --do_train
    
    if os.path.exists("output/soft_gen/checkpoint_soft_best.pt"):
        !cp outputs/soft_gen/checkpoint_soft_best.pt models/soft
    else:
        print("COULD NOT GET BEST MODEL")
        !cp outputs/soft_gen/checkpoint_soft_other.pt models/soft/checkpoint_soft_best.pt

# RUN CUSTOM DATASETS HERE (should be in data directory)
# --test_data "ER-reddit-test.csv"
if mode == 2 or mode == 3:
    !bash bash/run_generation.sh \
             --glove "resources/glove.6B.300d.txt" \
             --gpu_id 0 \
             --mode soft \
             --roberta roberta-large \
             --baseline "resources/roberta-large.tsv" \
             --do_test

/kaggle/working/EmpDialogue_RecEC/soft_gen
begin training
USING cuda
tokenizer_config.json: 100%|█████████████████| 25.0/25.0 [00:00<00:00, 69.1kB/s]
config.json: 100%|█████████████████████████████| 482/482 [00:00<00:00, 3.10MB/s]
vocab.json: 100%|████████████████████████████| 899k/899k [00:00<00:00, 6.71MB/s]
merges.txt: 100%|████████████████████████████| 456k/456k [00:00<00:00, 3.43MB/s]
tokenizer.json: 100%|██████████████████████| 1.36M/1.36M [00:00<00:00, 9.49MB/s]
model.safetensors: 100%|████████████████████| 1.42G/1.42G [00:07<00:00, 193MB/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[2024-03-10 20:49:03,971 INFO] Vocab size: 38622
[2024-03-10 20:49:03,972 INFO] EmotionVocab Size: 32
[2024-03-10 20:49:10,790 INFO] Training data size: 

In [7]:
torch.cuda.empty_cache()

mode = 3  # 1: train, 2: test, 3: both

if mode == 1 or mode == 3:
    !bash bash/run_generation.sh \
             --glove "resources/glove.6B.300d.txt" \
             --gpu_id 0 \
             --mode hard \
             --roberta roberta-large \
             --baseline "resources/roberta-large.tsv" \
             --do_train
    
    if os.path.exists("output/hard_gen/checkpoint_hard_best.pt"):
        !cp outputs/hard_gen/checkpoint_hard_best.pt models/hard
    else:
        print("COULD NOT GET BEST MODEL")
        !cp outputs/hard_gen/checkpoint_hard_other.pt models/hard/checkpoint_hard_best.pt

# RUN CUSTOM DATASETS HERE (should be in data directory)
# --test_data "ER-reddit-test.csv"
if mode == 2 or mode == 3:
    !bash bash/run_generation.sh \
             --glove "resources/glove.6B.300d.txt" \
             --gpu_id 0 \
             --mode hard \
             --roberta roberta-large \
             --baseline "resources/roberta-large.tsv" \
             --do_test

UnboundLocalError: local variable 'child' referenced before assignment